# Purpose
- examine probabilities of weekly options expiring in/out of the money
- seek to sell option premiums with low probability of ITM expiration

In [1]:
import sys
sys.path.append(<PATH>)
import tda_data
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import datetime
from scipy import stats
import pytz

pd.set_option('display.max_columns', 500)

today = datetime.datetime.now(pytz.timezone("US/Eastern")).date()
friday = today + datetime.timedelta( (4-today.weekday()) % 7 )
remain_days = (friday-today).days

In [2]:
tickers = ['TSLA', 'TQQQ', 'BITO']

In [17]:
# PARAMETERS: run_model, min_perc_gain_requirement, min_perc_OTM_expir

# conservative
run_con = True
con_min_pct = 0.002
CON_MIN_PR = 0.8

# target
run_targ = True
targ_pct = 0.006
TARG_MIN_PR = 0

# growth
run_grw = True
grw_min_pct = 0.01
GRW_MIN_PR = 0

In [ ]:
%cd <PATH>

weekly_dict = {}
last_price = {}

for ticker in tickers:
    print(ticker)
    weekly_dict[ticker] = tda_data.weekly_adj(ticker)
    weekly_dict[ticker]['week_change'] = weekly_dict[ticker]['close'] / weekly_dict[ticker]['open'] - 1
    last_price[ticker] = weekly_dict[ticker]['close'].iloc[-1]

In [ ]:
for ticker in tickers:
    print('\n=================================================')
    print(ticker)
    plt.hist(weekly_dict[ticker]['week_change'], bins=20)
    plt.title(ticker)
    plt.show()
    display(weekly_dict[ticker]['week_change'].describe())
    print('1st quantile', round(weekly_dict[ticker]['week_change'].quantile(0.01), 4))
    print('2.5th quantile', round(weekly_dict[ticker]['week_change'].quantile(0.025), 4))
    print('5th quantile', round(weekly_dict[ticker]['week_change'].quantile(0.05), 4))
    print('10th quantile', round(weekly_dict[ticker]['week_change'].quantile(0.1), 4))

In [19]:
contract_type = 'put'
option_dict = {}
ordered_all_tickers = tickers.copy()

for ticker in tickers:
    print(ticker)
    try:
        option_dict[ticker] = tda_data.option_chain(ticker, contract_type, friday, remain_days)
    except:
        print(ticker, 'SAME-WEEK CONTRACT UNAVAILABLE')
        unordered_success_tickers = list(set(tickers)-{ticker})
        tickers = [x for x in ordered_all_tickers if x in unordered_success_tickers]
    
# option_dict

TSLA
TQQQ
BITO


# Target model
- target profit, min PR, rank by PR

In [20]:
if run_targ:
    
    CP_MIN_PR, D_MIN_PR = TARG_MIN_PR, TARG_MIN_PR

    print('Target model')
    master_df = pd.DataFrame()
    for ticker in tickers:

        # print('\n==========================================')
        # print(ticker, last_price[ticker])
        temp_df = option_dict[ticker].copy(deep=True)
        temp_df = temp_df[(temp_df['mark']!=0) & (temp_df['bid']!=0) & (temp_df['ask']!=0)]
        # temp_df = temp_df[(temp_df['last']>=temp_df['bid']) & (temp_df['last']<=temp_df['ask'])]
        temp_df = temp_df[temp_df['strike'] < last_price[ticker]]

        temp_df['PL_last'] = temp_df['last'] / temp_df['strike']
        temp_df['PL_mark'] = temp_df['mark'] / temp_df['strike']
        temp_df['PL_bid'] = temp_df['bid'] / temp_df['strike']
        temp_df['change'] = temp_df['strike'] / last_price[ticker] - 1
        temp_df['change_percentile'] = [stats.percentileofscore(weekly_dict[ticker]\
                [weekly_dict[ticker]['week_change'] < 0]['week_change'], x) for x in temp_df['change']]    
        temp_df['change_percentile'] /= 100
        temp_df['delta'] *= -1

        temp_df['change_percentile'] = 1 - temp_df['change_percentile']
        temp_df.loc[temp_df['delta']=='', 'delta'] = 0
        temp_df['delta'] = 1 - temp_df['delta']
        temp_df['PR_AVG'] = (temp_df['change_percentile'] + temp_df['delta'] ) / 2

        E_X_cols = []
        for price_type in ['bid']:
            for change_type in ['PR_AVG']:
                temp_df[f'E_X_{price_type}_{change_type}'] = temp_df[f'PL_{price_type}'] * temp_df[change_type]
                E_X_cols.append(f'E_X_{price_type}_{change_type}')

        temp_df['distance_from_targ_pct'] = (targ_pct - temp_df['PL_mark']).abs()
        temp_df = temp_df[temp_df['distance_from_targ_pct']==temp_df['distance_from_targ_pct'].min()]
        temp_df = temp_df[(temp_df['change_percentile'] >= CP_MIN_PR) & (temp_df['delta'] >= D_MIN_PR)]

        temp_df['ticker'] = ticker
        temp_df['underlying_last'] = last_price[ticker]
        temp_df['capital_required'] = temp_df['strike'] * 100
        temp_df = temp_df[['ticker','strike','capital_required','PL_bid','PR_AVG','change',\
                           'underlying_last','strike','volatility','bid','ask','change_percentile'\
                           ,'delta'] + E_X_cols]

        master_df = pd.concat([master_df, temp_df])

    target_model = master_df.sort_values(by=['PR_AVG','PL_bid'], ascending=False).reset_index(drop=True)
    display(target_model)
    
else:
    target_model = None

Target model


,ticker,strike,capital_required,PL_bid,PR_AVG,change,underlying_last,strike,volatility,bid,ask,change_percentile,delta,E_X_bid_PR_AVG
0,BITO,10.00,1000.0,0.004000,0.966000,-0.157540,11.87,10.00,139.954,0.04,0.05,1.000000,0.932,0.003864
1,TQQQ,21.00,2100.0,0.006190,0.932950,-0.151515,24.75,21.00,149.335,0.13,0.14,0.953901,0.912,0.005775
2,TSLA,293.33,29333.0,0.005489,0.666774,-0.043468,306.66,293.33,59.103,1.61,1.64,0.520548,0.813,0.003660


# Conservative PR model
- conservative profit, min PR, rank by PR

In [21]:
if run_con:

    CP_MIN_PR, D_MIN_PR = CON_MIN_PR, CON_MIN_PR

    print('Conservative PR model')
    master_df = pd.DataFrame()
    for ticker in tickers:

        # print('\n==========================================')
        # print(ticker, last_price[ticker])
        temp_df = option_dict[ticker].copy(deep=True)
        temp_df = temp_df[(temp_df['mark']!=0) & (temp_df['bid']!=0) & (temp_df['ask']!=0)]
        # temp_df = temp_df[(temp_df['last']>=temp_df['bid']) & (temp_df['last']<=temp_df['ask'])]
        temp_df = temp_df[temp_df['strike'] < last_price[ticker]]

        temp_df['PL_last'] = temp_df['last'] / temp_df['strike']
        temp_df['PL_mark'] = temp_df['mark'] / temp_df['strike']
        temp_df['PL_bid'] = temp_df['bid'] / temp_df['strike']
        temp_df['change'] = temp_df['strike'] / last_price[ticker] - 1
        temp_df['change_percentile'] = [stats.percentileofscore(weekly_dict[ticker][weekly_dict[ticker]['week_change'] < 0]['week_change'], x) for x in temp_df['change']]    
        temp_df['change_percentile'] /= 100
        temp_df['delta'] *= -1

        temp_df['change_percentile'] = 1 - temp_df['change_percentile']
        temp_df.loc[temp_df['delta']=='', 'delta'] = 0
        temp_df['delta'] = 1 - temp_df['delta']
        temp_df['PR_AVG'] = (temp_df['change_percentile'] + temp_df['delta'] ) / 2

        E_X_cols = []
        for price_type in ['bid']:
            for change_type in ['PR_AVG']:
                temp_df[f'E_X_{price_type}_{change_type}'] = temp_df[f'PL_{price_type}'] * temp_df[change_type]
                E_X_cols.append(f'E_X_{price_type}_{change_type}')

        temp_df = temp_df[(temp_df['change_percentile'] >= CP_MIN_PR) & (temp_df['delta'] >= D_MIN_PR)]
        temp_df = temp_df[(temp_df['PL_bid'] >= con_min_pct)]

        temp_df['ticker'] = ticker
        temp_df['underlying_last'] = last_price[ticker]
        temp_df['capital_required'] = temp_df['strike'] * 100
        temp_df = temp_df[['ticker','strike','capital_required','PL_bid','PR_AVG','change','underlying_last','volatility','bid','ask','change_percentile','delta'] + E_X_cols]

        master_df = pd.concat([master_df, temp_df])

    conservative_model = master_df.sort_values(by=['ticker','PR_AVG','PL_bid'], ascending=False).reset_index(drop=True)
    display(conservative_model)
    
else:
    conservative_model = None

Conservative PR model


,ticker,strike,capital_required,PL_bid,PR_AVG,change,underlying_last,volatility,bid,ask,change_percentile,delta,E_X_bid_PR_AVG
0,TQQQ,19.5,1950.0,0.002051,0.979954,-0.212121,24.75,157.458,0.04,0.05,0.992908,0.967,0.002010
1,TQQQ,20.0,2000.0,0.003000,0.962816,-0.191919,24.75,154.205,0.06,0.07,0.971631,0.954,0.002888
2,TQQQ,20.5,2050.0,0.004390,0.946723,-0.171717,24.75,151.917,0.09,0.10,0.957447,0.936,0.004156
3,TQQQ,21.0,2100.0,0.006190,0.932950,-0.151515,24.75,149.335,0.13,0.14,0.953901,0.912,0.005775
4,TQQQ,21.5,2150.0,0.008837,0.899720,-0.131313,24.75,148.333,0.19,0.20,0.918440,0.881,0.007951
5,TQQQ,22.0,2200.0,0.012273,0.855397,-0.111111,24.75,147.481,0.27,0.28,0.868794,0.842,0.010498
6,BITO,9.5,950.0,0.002105,0.981000,-0.199663,11.87,152.331,0.02,0.03,1.000000,0.962,0.002065
7,BITO,10.0,1000.0,0.004000,0.966000,-0.157540,11.87,139.954,0.04,0.05,1.000000,0.932,0.003864
8,BITO,10.5,1050.0,0.006667,0.893613,-0.115417,11.87,124.847,0.07,0.08,0.903226,0.884,0.005957


# Growth PL model
- growth profit, min PR, rank by PL

In [22]:
if run_grw:

    CP_MIN_PR, D_MIN_PR = GRW_MIN_PR, GRW_MIN_PR

    print('Growth PL model')
    master_df = pd.DataFrame()
    for ticker in tickers:

        # print('\n==========================================')
        # print(ticker, last_price[ticker])
        temp_df = option_dict[ticker].copy(deep=True)
        temp_df = temp_df[(temp_df['mark']!=0) & (temp_df['bid']!=0) & (temp_df['ask']!=0)]
        # temp_df = temp_df[(temp_df['last']>=temp_df['bid']) & (temp_df['last']<=temp_df['ask'])]
        temp_df = temp_df[temp_df['strike'] < last_price[ticker]]

        temp_df['PL_last'] = temp_df['last'] / temp_df['strike']
        temp_df['PL_mark'] = temp_df['mark'] / temp_df['strike']
        temp_df['PL_bid'] = temp_df['bid'] / temp_df['strike']
        temp_df['change'] = temp_df['strike'] / last_price[ticker] - 1
        temp_df['change_percentile'] = [stats.percentileofscore(weekly_dict[ticker][weekly_dict[ticker]['week_change'] < 0]['week_change'], x) for x in temp_df['change']]    
        temp_df['change_percentile'] /= 100
        temp_df['delta'] *= -1

        temp_df['change_percentile'] = 1 - temp_df['change_percentile']
        temp_df.loc[temp_df['delta']=='', 'delta'] = 0
        temp_df['delta'] = 1 - temp_df['delta']
        temp_df['PR_AVG'] = (temp_df['change_percentile'] + temp_df['delta'] ) / 2

        E_X_cols = []
        for price_type in ['bid']:
            for change_type in ['PR_AVG']:
                temp_df[f'E_X_{price_type}_{change_type}'] = temp_df[f'PL_{price_type}'] * temp_df[change_type]
                E_X_cols.append(f'E_X_{price_type}_{change_type}')

        temp_df = temp_df[(temp_df['change_percentile'] >= CP_MIN_PR) & (temp_df['delta'] >= D_MIN_PR)]
        temp_df = temp_df[(temp_df['PL_bid'] >= grw_min_pct)]

        temp_df['ticker'] = ticker
        temp_df['underlying_last'] = last_price[ticker]
        temp_df['capital_required'] = temp_df['strike'] * 100
        temp_df = temp_df[['ticker','strike','capital_required','PL_bid','PR_AVG','change','underlying_last','volatility','bid','ask','change_percentile','delta'] + E_X_cols]

        master_df = pd.concat([master_df, temp_df])

    growth_model = master_df.sort_values(by=['PL_bid','PR_AVG'], ascending=False).reset_index(drop=True)
    display(growth_model)
    
else:
    growth_model = None

Growth PL model


,ticker,strike,capital_required,PL_bid,PR_AVG,change,underlying_last,volatility,bid,ask,change_percentile,delta,E_X_bid_PR_AVG
0,TQQQ,24.50,2450.0,0.043265,0.368152,-0.010101,24.75,143.903,1.06,1.10,0.177305,0.559,0.015928
1,TQQQ,24.00,2400.0,0.035417,0.501716,-0.030303,24.75,144.902,0.85,0.87,0.379433,0.624,0.017769
2,TQQQ,23.50,2350.0,0.028511,0.625688,-0.050505,24.75,146.967,0.67,0.69,0.567376,0.684,0.017839
3,TQQQ,23.00,2300.0,0.022174,0.720791,-0.070707,24.75,146.969,0.51,0.52,0.698582,0.743,0.015983
4,BITO,11.50,1150.0,0.019130,0.508919,-0.031171,11.87,96.603,0.22,0.24,0.354839,0.663,0.009736
5,TSLA,305.00,30500.0,0.017541,0.303822,-0.005413,306.66,58.256,5.35,5.45,0.061644,0.546,0.005329
6,TQQQ,22.50,2250.0,0.016444,0.793390,-0.090909,24.75,146.980,0.37,0.39,0.790780,0.796,0.013047
7,TSLA,303.33,30333.0,0.015330,0.356644,-0.010859,306.66,58.617,4.65,4.70,0.123288,0.590,0.005467
8,TSLA,301.67,30167.0,0.013094,0.411890,-0.016272,306.66,58.606,3.95,4.05,0.191781,0.632,0.005393
9,TQQQ,22.00,2200.0,0.012273,0.855397,-0.111111,24.75,147.481,0.27,0.28,0.868794,0.842,0.010498


# All models

In [23]:
print('\n====================\ntarget_model')
display(target_model)

print('\n====================\nconservative_model')
display(conservative_model)

print('\n====================\ngrowth_model')
display(growth_model)


target_model


,ticker,strike,capital_required,PL_bid,PR_AVG,change,underlying_last,strike,volatility,bid,ask,change_percentile,delta,E_X_bid_PR_AVG
0,BITO,10.00,1000.0,0.004000,0.966000,-0.157540,11.87,10.00,139.954,0.04,0.05,1.000000,0.932,0.003864
1,TQQQ,21.00,2100.0,0.006190,0.932950,-0.151515,24.75,21.00,149.335,0.13,0.14,0.953901,0.912,0.005775
2,TSLA,293.33,29333.0,0.005489,0.666774,-0.043468,306.66,293.33,59.103,1.61,1.64,0.520548,0.813,0.003660



conservative_model


,ticker,strike,capital_required,PL_bid,PR_AVG,change,underlying_last,volatility,bid,ask,change_percentile,delta,E_X_bid_PR_AVG
0,TQQQ,19.5,1950.0,0.002051,0.979954,-0.212121,24.75,157.458,0.04,0.05,0.992908,0.967,0.002010
1,TQQQ,20.0,2000.0,0.003000,0.962816,-0.191919,24.75,154.205,0.06,0.07,0.971631,0.954,0.002888
2,TQQQ,20.5,2050.0,0.004390,0.946723,-0.171717,24.75,151.917,0.09,0.10,0.957447,0.936,0.004156
3,TQQQ,21.0,2100.0,0.006190,0.932950,-0.151515,24.75,149.335,0.13,0.14,0.953901,0.912,0.005775
4,TQQQ,21.5,2150.0,0.008837,0.899720,-0.131313,24.75,148.333,0.19,0.20,0.918440,0.881,0.007951
5,TQQQ,22.0,2200.0,0.012273,0.855397,-0.111111,24.75,147.481,0.27,0.28,0.868794,0.842,0.010498
6,BITO,9.5,950.0,0.002105,0.981000,-0.199663,11.87,152.331,0.02,0.03,1.000000,0.962,0.002065
7,BITO,10.0,1000.0,0.004000,0.966000,-0.157540,11.87,139.954,0.04,0.05,1.000000,0.932,0.003864
8,BITO,10.5,1050.0,0.006667,0.893613,-0.115417,11.87,124.847,0.07,0.08,0.903226,0.884,0.005957



growth_model


,ticker,strike,capital_required,PL_bid,PR_AVG,change,underlying_last,volatility,bid,ask,change_percentile,delta,E_X_bid_PR_AVG
0,TQQQ,24.50,2450.0,0.043265,0.368152,-0.010101,24.75,143.903,1.06,1.10,0.177305,0.559,0.015928
1,TQQQ,24.00,2400.0,0.035417,0.501716,-0.030303,24.75,144.902,0.85,0.87,0.379433,0.624,0.017769
2,TQQQ,23.50,2350.0,0.028511,0.625688,-0.050505,24.75,146.967,0.67,0.69,0.567376,0.684,0.017839
3,TQQQ,23.00,2300.0,0.022174,0.720791,-0.070707,24.75,146.969,0.51,0.52,0.698582,0.743,0.015983
4,BITO,11.50,1150.0,0.019130,0.508919,-0.031171,11.87,96.603,0.22,0.24,0.354839,0.663,0.009736
5,TSLA,305.00,30500.0,0.017541,0.303822,-0.005413,306.66,58.256,5.35,5.45,0.061644,0.546,0.005329
6,TQQQ,22.50,2250.0,0.016444,0.793390,-0.090909,24.75,146.980,0.37,0.39,0.790780,0.796,0.013047
7,TSLA,303.33,30333.0,0.015330,0.356644,-0.010859,306.66,58.617,4.65,4.70,0.123288,0.590,0.005467
8,TSLA,301.67,30167.0,0.013094,0.411890,-0.016272,306.66,58.606,3.95,4.05,0.191781,0.632,0.005393
9,TQQQ,22.00,2200.0,0.012273,0.855397,-0.111111,24.75,147.481,0.27,0.28,0.868794,0.842,0.010498
